In [66]:
import os
import pandas as pd
import json
from openai import OpenAI
import openai
from dotenv import load_dotenv

# Load environment variables from .env 
load_dotenv()

# Access the API key from environment variables
api_key = os.getenv("OPENAI_API_KEY")
# Set the OpenAI API key
os.environ['OPENAI_API_KEY'] = api_key
#API Key to OpenAI Library
openai.api_key = os.getenv("OPENAI_API_KEY")

In [58]:
train_df = pd.read_parquet("hf://datasets/kuroschin/airline-customer-support/data/train-00000-of-00001.parquet")

In [67]:
output_file = 'training.jsonl'
with open(output_file, 'w') as f:
    for _, row in train_df.iterrows():
        json_line = json.dumps({
        "messages": [
            {"role": "system", "content": "You are a helpful assistant as FAQ support."},
            {"role": "user", "content": row["input"]},
            {"role": "assistant", "content": row["output"]}
            ]
        })
        f.write(json_line + '\n')

print(f"Dataset converted and saved to {output_file}")

Dataset converted and saved to training.jsonl
